## Import some helpful libraries

In [1]:
print("Version control:\n")
import os     # operating system interfaces
import fnmatch
import random
import datetime
import numpy as np; print("Numpy\t\t", np.__version__)
import matplotlib as mpl; print("matplotlib\t", mpl.__version__)
import matplotlib.pyplot as plt
import nibabel as nib; print("NiBabel\t\t", nib.__version__)
from nibabel.testing import data_path
import math
import pandas as pd; print("Pandas\t\t", pd.__version__)
import sys
import imageio; print("imageio\t\t", imageio.__version__)
import h5py; print("H5py\t\t", h5py.__version__)
import sklearn; print("Scikit-learn\t", sklearn.__version__)
import skimage; print("Scikit-image\t", skimage.__version__)
import tensorflow as tf; print("TensorFlow\t", tf.__version__)
import keras; print("Keras\t\t", keras.__version__)
from tensorflow.keras import models, Input, Model
from tensorflow.keras.layers import Dense, Conv3D, Conv3DTranspose, MaxPooling3D, UpSampling3D
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import TensorBoard

Version control:

Numpy		 1.18.5
matplotlib	 3.3.2
NiBabel		 3.2.0
Pandas		 1.1.4
imageio		 2.9.0
H5py		 2.10.0
Scikit-learn	 0.23.2
Scikit-image	 0.17.2
TensorFlow	 2.3.1
Keras		 2.3.1


Using TensorFlow backend.


## Loading Dataset:Validation and Training Dataset (randomly)

In [2]:
## Loading all datasets from path directory

# example_filename = os.path.join(data_path, 'example4d.nii.gz')
print(os.getcwd())     # '/Users/shossein/GitHub/DTI_Clustering'
dirpath = os.chdir("/Users/shossein/anaconda3/envs/m36/lib/python3.6/site-packages/nibabel/tests/data/DTI")
print("\nDirectory is changed into:\n", os.getcwd(), "\n") 
sample_all = []
for sample in fnmatch.filter(os.listdir(dirpath), 'Pat_*_1-post.nii.gz'):
    sample_all.append(sample)
sample_num = len(sample_all)
print("There are", sample_num, "samples.\n")
sample_all.sort()
# print(*sample_all, sep='\t')

/Users/shossein/GitHub/DTI_Clustering

Directory is changed into:
 /Users/shossein/anaconda3/envs/m36/lib/python3.6/site-packages/nibabel/tests/data/DTI 

There are 124 samples.



In [3]:
## Deviding all samples into two groups: Training and Validation datasets randomly

sample_val = random.sample(sample_all, 24)
sample_val.sort()
print(len(sample_val), "Validation samples are:\n")
print(*sample_val, sep='\t')

sample_val_subset = sample_val [0:3]
print("\n\n", len(sample_val_subset), " subset of Validation samples are:\n")
print(*sample_val_subset, sep='\t')

sample_train = [x for x in sample_all if x not in sample_val]
sample_train.sort()
print("\n--------------------------------------------------------------------------------\n", len(sample_train), "Training samples are:\n")
print(*sample_train, sep='\t')

sample_train_subset = sample_train [0:5]
print("\n\n", len(sample_train_subset), " subset of Validation samples are:\n")
print(*sample_train_subset, sep='\t')

24 Validation samples are:

Pat_109_1-post.nii.gz	Pat_114_1-post.nii.gz	Pat_118_1-post.nii.gz	Pat_11_1-post.nii.gz	Pat_129_1-post.nii.gz	Pat_130_1-post.nii.gz	Pat_131_1-post.nii.gz	Pat_137_1-post.nii.gz	Pat_138_1-post.nii.gz	Pat_139_1-post.nii.gz	Pat_144_1-post.nii.gz	Pat_145_1-post.nii.gz	Pat_22_1-post.nii.gz	Pat_24_1-post.nii.gz	Pat_25_1-post.nii.gz	Pat_26_1-post.nii.gz	Pat_60_1-post.nii.gz	Pat_65_1-post.nii.gz	Pat_66_1-post.nii.gz	Pat_76_1-post.nii.gz	Pat_83_1-post.nii.gz	Pat_96_1-post.nii.gz	Pat_97_1-post.nii.gz	Pat_99_1-post.nii.gz


 3  subset of Validation samples are:

Pat_109_1-post.nii.gz	Pat_114_1-post.nii.gz	Pat_118_1-post.nii.gz

--------------------------------------------------------------------------------
 100 Training samples are:

Pat_100_1-post.nii.gz	Pat_101_1-post.nii.gz	Pat_102_1-post.nii.gz	Pat_103_1-post.nii.gz	Pat_107_1-post.nii.gz	Pat_108_1-post.nii.gz	Pat_10_1-post.nii.gz	Pat_110_1-post.nii.gz	Pat_113_1-post.nii.gz	Pat_115_1-post.nii.gz	Pat_116_1-post.nii.gz

## Combine datasets together

In [4]:
## Combine all Training dateset together

data_combined = np.zeros((1, 182, 218, 182, 1))

sample_train = sample_train_subset            # To test in CPU based computer

for sample in sample_train_subset:
#     print(sample)
    img = nib.load(sample)
    img_np = np.asanyarray(img.dataobj)         # Convert data into Numpy array format
    data_64 = np.float64(img_np)                # Convert numpayed data back into the original float64
    data_reshape = np.reshape(data_64, (1, 182, 218, 182, 1))
    data_combined = np.concatenate((data_combined, data_reshape), axis=0)
data_combined = np.delete(data_combined, 0, axis=0)
print("Shape of training dataset:", data_combined.shape)
train_data = data_combined

Shape of training dataset: (5, 182, 218, 182, 1)


In [6]:
## Showing one or all Training samples in three dimension (one middle slice per each dimension)

def show_slices(slices):
    fig, axes = plt.subplots(1, len(slices), figsize=(10,5))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="hot", origin="upper") # hot, Greys, gray
for m in range(data_combined.shape[0]):
    slice_0 = data_combined[m, 91, :, :, 0]
    slice_1 = data_combined[m, :, 109, :, 0]
    slice_2 = data_combined[m, :, :, 91, 0]
#     show_slices([slice_0, slice_1, slice_2])
    plt.suptitle(sample_train[m], x=0.5, y=0.9)

<Figure size 432x288 with 0 Axes>

In [8]:
## Rescale Training dataset into [0:1]

sample_spec = 2      # A specific sample to show the min and max of that 
print("Range of a sample of original training data:\tMin=", np.min(train_data[sample_spec, :, :, :, 0]), ",   Max=", np.max(train_data[sample_spec, :, :, :, 0]), "\n")
for m in range(train_data.shape[0]):
    for x in range(train_data.shape[1]):
        for y in range(train_data.shape[2]):
            for z in range(train_data.shape[3]):
                if train_data[m,x,y,z] <= 0: train_data[m,x,y,z] = 0
                if train_data[m,x,y,z] > 1: train_data[m,x,y,z] = 1
print("Range of a sample of original data:\t\tMin=", np.min(train_data[sample_spec, :, :, :, 0]), ",   Max=", np.max(train_data[sample_spec, :, :, :, 0]), "\n")

Range of a sample of original training data:	Min= -0.0010085757821798325 ,   Max= 0.9992789626121521 

Range of a sample of original data:		Min= 0.0 ,   Max= 0.9992789626121521 



In [9]:
## Combine all Validation dateset together

data_combined = np.zeros((1, 182, 218, 182, 1))

sample_val = sample_val_subset            # To test in CPU based computer

for sample in sample_val:
#     print(sample)
    img = nib.load(sample)
    img_np = np.asanyarray(img.dataobj)         # Convert data into Numpy array format
    data_64 = np.float64(img_np)                # Convert numpayed data back into the original float64
    data_reshape = np.reshape(data_64, (1, 182, 218, 182, 1))
    data_combined = np.concatenate((data_combined, data_reshape), axis=0)
data_combined = np.delete(data_combined, 0, axis=0)
print("Shape of validation dataset:", data_combined.shape)
valid_data = data_combined

Shape of validation dataset: (3, 182, 218, 182, 1)


In [10]:
## Showing one or all Validation samples in three dimension (one middle slice per each dimension)

for m in range(data_combined.shape[0]):
    slice_0 = data_combined[m, 91, :, :, 0]
    slice_1 = data_combined[m, :, 109, :, 0]
    slice_2 = data_combined[m, :, :, 91, 0]
#     show_slices([slice_0, slice_1, slice_2])
    plt.suptitle(sample_val[m], x=0.5, y=1)

<Figure size 432x288 with 0 Axes>

In [11]:
## Rescale Validation dataset into [0:1]

sample_spec = 2      # A specific sample to show the min and max of that 

print("Range of a sample of original validation data:\tMin=", np.min(valid_data[sample_spec, :, :, :, 0]), ",   Max=", np.max(valid_data[sample_spec, :, :, :, 0]), "\n")
for m in range(valid_data.shape[0]):
    for x in range(valid_data.shape[1]):
        for y in range(valid_data.shape[2]):
            for z in range(valid_data.shape[3]):
                if valid_data[m,x,y,z] <= 0: valid_data[m,x,y,z] = 0
                if valid_data[m,x,y,z] > 1: valid_data[m,x,y,z] = 1
print("Range of a sample of original data:\t\tMin=", np.min(valid_data[sample_spec, :, :, :, 0]), ",   Max=", np.max(valid_data[sample_spec, :, :, :, 0]), "\n")

Range of a sample of original validation data:	Min= 0.0 ,   Max= 0.9882298111915588 

Range of a sample of original data:		Min= 0.0 ,   Max= 0.9882298111915588 



### Test for shape and one sample

In [13]:
train_data.shape

(5, 182, 218, 182, 1)

In [14]:
train_data[[1],[100],[100],[100]]

array([[0.69851756]])

In [15]:
valid_data.shape

(3, 182, 218, 182, 1)

In [16]:
valid_data[[1],[100],[100],[100]]

array([[0.73677564]])

# Reshape

In [17]:
print(os.getcwd())

/Users/shossein/anaconda3/envs/m36/lib/python3.6/site-packages/nibabel/tests/data/DTI


In [18]:
dirpath = os.chdir("/Users/shossein/GitHub/DTI_Clustering")
print("\nDirectory is changed into:\n", os.getcwd(), "\n")


Directory is changed into:
 /Users/shossein/GitHub/DTI_Clustering 



In [22]:
train_data_reshaped = train_data.reshape(5,7221032)
valid_data_reshaped = valid_data.reshape(3,7221032)
print("train_data reshaped to:", train_data_reshaped.shape, " and valid_data reshaped to :", valid_data_reshaped.shape)

train_data reshaped to: (5, 7221032)  and valid_data reshaped to : (3, 7221032)


In [23]:
np.savetxt("train_data.csv", train_data_reshaped, delimiter=",")
np.savetxt("valid_data.csv", valid_data_reshaped, delimiter=",")

In [24]:
train_data_loaded = np.loadtxt("train_data.csv", delimiter=",").reshape((5, 182, 218, 182, 1))
valid_data_loaded = np.loadtxt("valid_data.csv", delimiter=",").reshape((3, 182, 218, 182, 1))

In [25]:
train_data_loaded.shape

(5, 182, 218, 182, 1)

In [26]:
train_data_loaded[[1],[100],[100],[100]]

array([[0.69851756]])

In [27]:
valid_data_loaded.shape

(3, 182, 218, 182, 1)

In [39]:
valid_data_loaded[[1],[100],[100],[100]]

array([[0.73677564]])

In [32]:
assert valid_data[[1],[100],[100],[100]] == valid_data_loaded[[1],[100],[100],[100]]

In [33]:
assert train_data[[1],[100],[100],[100]] == train_data_loaded[[1],[100],[100],[100]]

In [35]:
assert train_data.all() == train_data_loaded.all()

In [36]:
assert valid_data.all() == valid_data_loaded.all()

## Model Design

In [40]:
## Define parameters:

IMAGE_HEIGHT = train_data_loaded.shape[1]
IMAGE_WIDTH = train_data_loaded.shape[2]
IMAGE_DEPTH = train_data_loaded.shape[3]
batch_size = 32
data_shape = [1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
input_shape = [batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
print("input-layer shape:", input_shape)

## Encoder
input_img = Input(shape=(182, 218, 182, 1))
x = Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu', name='Conv1')(input_img)
x = MaxPooling3D(pool_size=(13, 13, 13), padding='same')(x)
x = Conv3D(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu', name='Conv2')(x)
encoded = MaxPooling3D(pool_size=(7, 7, 7), padding='same')(x)
## at this point the representation is (2, 3, 2, 8) i.e. 96-dimensional instead of 7,221,032

## Decoder
x = Conv3DTranspose(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu', name='DeConv3')(encoded)
x = UpSampling3D(size=(7, 6, 7))(x)
x = Conv3DTranspose(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu', name='DeConv4')(x)
x = UpSampling3D(size=(13, 12, 13))(x)
decoded = Conv3DTranspose(filters=1, kernel_size=(1, 3, 1), padding='valid', activation='sigmoid', name='DeConv5')(x)

autoencoder = Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
autoencoder.summary()

input-layer shape: [32, 182, 218, 182, 1]
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 182, 218, 182, 1) 0         
_________________________________________________________________
Conv1 (Conv3D)               (None, 182, 218, 182, 16) 448       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 14, 17, 14, 16)    0         
_________________________________________________________________
Conv2 (Conv3D)               (None, 14, 17, 14, 8)     3464      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 2, 3, 2, 8)        0         
_________________________________________________________________
DeConv3 (Conv3DTranspose)    (None, 2, 3, 2, 4)        868       
_________________________________________________________________
up_sampling3

## Model Training

In [41]:
## Tensorboard

# %load_ext tensorboard
# %reload_ext tensorboard
# logdir = os.path.join("CAE_logs", datetime.datetime.now().strftime("%Y_%m_%d____%H_%M_%S"))
# tb_callback = TensorBoard(logdir, histogram_freq=1)

## Midel Fit
# autoencoder.fit(train_data, train_data, epochs=3, batch_size=batch_size, shuffle=True, validation_data=(valid_data, valid_data), callbacks=[tb_callback], verbose=1)
autoencoder.fit(train_data_loaded, train_data_loaded, epochs=3, batch_size=batch_size, shuffle=True, validation_data=(valid_data_loaded, valid_data_loaded), verbose=1)
autoencoder.save_weights("CAE_weights.hdf5")
# autoencoder.load_weights("CAE_weights.hdf5")     # loading weights

Epoch 1/3
1/1 [==============================] - 8s 8s/step - loss: 0.0000e+00 - accuracy: 0.3087 - val_loss: 0.0000e+00 - val_accuracy: 0.3026
Epoch 2/3
1/1 [==============================] - 10s 10s/step - loss: 0.0000e+00 - accuracy: 0.3087 - val_loss: 0.0000e+00 - val_accuracy: 0.3026
Epoch 3/3
1/1 [==============================] - 9s 9s/step - loss: 0.0000e+00 - accuracy: 0.3087 - val_loss: 0.0000e+00 - val_accuracy: 0.3026


In [42]:
# tensorboard --logdir=CAE_logs